# Hyperparameter Tuning

This files tests different parameters for the Random Forest classifier, and assess where the validation data reaches a maximum without overfitting.

In [15]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [16]:
ethnic_group_names = ['unknown', 'white', 'other', 'asian', 'hispanic_latino', 'black_african_american',
                      'unable_to_obtain', 'american_indian_alaska_native']
path = './data/preprocessing_IV/'

def read_train_val_test(name, path):
    train = pd.read_csv(path + name + '_train.csv')
    # NOTE: drope datetime columns for now, because sklearn does not support it
    train = train.drop(columns=['admittime', 'edregtime', 'emar_charttime', 'lab_charttime'])
    validate = pd.read_csv(path + name + '_validate.csv')
    # NOTE: drope datetime columns for now, because sklearn does not support it
    validate = validate.drop(columns=['admittime', 'edregtime', 'emar_charttime', 'lab_charttime'])
    test = pd.read_csv(path + name + '_test.csv')
    # NOTE: drope datetime columns for now, because sklearn does not support it
    test = test.drop(columns=['admittime', 'edregtime', 'emar_charttime', 'lab_charttime'])
    return train, validate, test

In [17]:
# split features and labels
def splitFeaturesLabels(df):
    X = df.copy()
    X.drop('has_kidney_issue', axis=1)
    # X = df[['anchor_age', 'anchor_year']]
    return X, df.has_kidney_issue

In [18]:
def tune(name, train, val):
    # Initiate empty df to store results
    results = pd.DataFrame({"ethnicity": [], "max_depth":[], "n_estimators":[], "accuracy":[]})

    # split labels and features
    # X = df[['hadm_id', 'subject_id', 'admittime', 'anchor_age', 'anchor_year', 'icd_code_count', 'emar_count', 'lab_count']]
    # y = df['has_kidney_issue']

    X_train, y_train = splitFeaturesLabels(train)
    X_val, y_val = splitFeaturesLabels(val)

    # split train and test
    # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

    # try different values for max_depth of the individual trees
    for depth in [6, 8, 10, 12, 14, 16, 18, 20]:
        # try different values for the number of trees in a forest
        for estimator in [10, 50, 100, 150, 200, 250, 300]:
            # train classifier
            clf = RandomForestClassifier(n_estimators=estimator, max_depth=depth, random_state=42)
            clf.fit(X_train, y_train)
            # predict labels
            y_pred = clf.predict(X_val)
            # calculate accuracy
            accuracy = accuracy_score(y_val, y_pred)
            # append results to df
            new_result = pd.DataFrame({"ethnicity": name, "max_depth":depth, "n_estimators":estimator, "accuracy":accuracy}, index=[0])
            results = pd.concat([results, new_result])
    
    # reset indexing
    results = results.reset_index(drop=True)

    # return
    return results

In [19]:
def getBestParams(df, results):
    return df.append(results.loc[results['accuracy'].idxmax()])

def tuneEachEthnicity(groups):
    all_results = pd.DataFrame({"ethnicity": [], "max_depth":[], "n_estimators":[], "accuracy":[]})
    for ethnicity in groups:
        train, val, test = read_train_val_test(ethnicity, path)
        results = tune(ethnicity, train, val)
        all_results = getBestParams(all_results, results)
    return all_results

In [20]:
def runToCSV():
    results = tuneEachEthnicity(ethnic_group_names)
    results.to_csv('hyperparameters.csv')

runToCSV()

C:\Users\annik\AppData\Local\Temp\ipykernel_15484\771118452.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return df.append(results.loc[results['accuracy'].idxmax()])
C:\Users\annik\AppData\Local\Temp\ipykernel_15484\771118452.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return df.append(results.loc[results['accuracy'].idxmax()])
C:\Users\annik\AppData\Local\Temp\ipykernel_15484\771118452.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return df.append(results.loc[results['accuracy'].idxmax()])
C:\Users\annik\AppData\Local\Temp\ipykernel_15484\771118452.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  return df.append

In [21]:
white = tune('white')
print("Highest accuracy:\n", white.loc[white['accuracy'].idxmax()])
ax = plt.axes(projection='3d')
ax.plot_trisurf(white["max_depth"], white["n_estimators"], white["accuracy"], cmap='viridis', edgecolor='none')
ax.set_title("white")
ax.set_xlabel("max_depth")
ax.set_ylabel("n_estimators")
ax.set_zlabel("accuracy")

TypeError: tune() missing 2 required positional arguments: 'train' and 'val'

In [ ]:
other = tune('other')
print("Highest accuracy:\n", other.loc[other['accuracy'].idxmax()])
ax = plt.axes(projection='3d')
ax.plot_trisurf(other["max_depth"], other["n_estimators"], other["accuracy"], cmap='viridis', edgecolor='none')

In [ ]:
asian = tune('asian')
print("Highest accuracy:\n", asian.loc[asian['accuracy'].idxmax()])
ax = plt.axes(projection='3d')
ax.plot_trisurf(asian["max_depth"], asian["n_estimators"], asian["accuracy"], cmap='viridis', edgecolor='none')

In [ ]:
hispanic_latino = tune('hispanic_latino')
print("Highest accuracy:\n", hispanic_latino.loc[hispanic_latino['accuracy'].idxmax()])
ax = plt.axes(projection='3d')
ax.plot_trisurf(hispanic_latino["max_depth"], hispanic_latino["n_estimators"], hispanic_latino["accuracy"], cmap='viridis', edgecolor='none')

In [ ]:
black_african_american = tune('black_african_american')
print("Highest accuracy:\n", black_african_american.loc[black_african_american['accuracy'].idxmax()])
ax = plt.axes(projection='3d')
ax.plot_trisurf(black_african_american["max_depth"], black_african_american["n_estimators"], black_african_american["accuracy"], cmap='viridis', edgecolor='none')

In [ ]:
unable_to_obtain = tune('unable_to_obtain')
print("Highest accuracy:\n", unable_to_obtain.loc[unable_to_obtain['accuracy'].idxmax()])
ax = plt.axes(projection='3d')
ax.plot_trisurf(unable_to_obtain["max_depth"], unable_to_obtain["n_estimators"], unable_to_obtain["accuracy"], cmap='viridis', edgecolor='none')

In [ ]:
american_indian_alaska_native = tune('american_indian_alaska_native')
print("Highest accuracy:\n", american_indian_alaska_native.loc[american_indian_alaska_native['accuracy'].idxmax()])
ax = plt.axes(projection='3d')
ax.plot_trisurf(american_indian_alaska_native["max_depth"], american_indian_alaska_native["n_estimators"], american_indian_alaska_native["accuracy"], cmap='viridis', edgecolor='none')